In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#this model is  inspired from 
#https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

#https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17



https://github.com/strongio/keras-bert/blob/d81d6cd21c7c191a766ae671ed040f4f8868b561/keras-bert.py#L155

In [0]:

##NOT FOR SYSTEMS
!unzip -P ****** -qq '/content/drive/My Drive/EnglishDatac/task_c_distant.zip' 

In [3]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.2MB/s 


In [4]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K



In [5]:

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np

Using TensorFlow backend.


In [0]:
# Initialize session
sess = tf.Session()

# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
max_seq_length = 256

In [0]:

def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


headers=['id','text','average_ind','average_grp','average_oth','std_ind','std_grp','std_oth']
taskc = pd.read_csv("task_c_distant_ann.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average_ind":convertToFloat,
                                                                                                    "average_grp":convertToFloat,
                                                                                                    "average_oth":convertToFloat,
                                                                                                    "std_ind":convertToFloat,
                                                                                                    "std_grp":convertToFloat,
                                                                                                    "std_oth":convertToFloat})
taskc=taskc[1:]

In [8]:
taskc.head()

,id,text,average_ind,average_grp,average_oth,std_ind,std_grp,std_oth
1,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.833432,0.076110,0.107765,0.208334,0.098937,0.138649
2,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.481062,0.367363,0.138841,0.345225,0.335924,0.083230
3,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.438813,0.268574,0.377573,0.182609,0.186880,0.254621
4,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.712995,0.123504,0.111130,0.248839,0.107572,0.067552
5,1159533742366633984,@USER @USER Fuck him better than his hoes,0.691414,0.146723,0.192282,0.204415,0.154818,0.104436


In [9]:
taskc.values

array([['1159533712079503361',
        '@USER Trump is a fucking idiot his dementia is getting worse',
        0.8334315660070198, ..., 0.20833409336419706,
        0.09893746284301566, 0.1386493230104102],
       ['1159533713044234241',
        '@USER HELL YES! His grinned and thumbs up are disgusting!',
        0.48106237178573397, ..., 0.3452248610539589,
        0.33592350092689993, 0.08323036242339713],
       ['1159533718345830400',
        "@USER Can't wait to see the shit show his death will bring.",
        0.4388127663192773, ..., 0.18260870386171185,
        0.18688048564468512, 0.25462109958562573],
       ...,
       ['1187636295306268673',
        'I meant tool but the point is women ain’t shit. 🙅🏻\u200d♂️🤷🏻\u200d♂️',
        0.36375786996300313, ..., 0.23943555651707127,
        0.16535075005267147, 0.09506541000311416],
       ['1187636299785744384', '@USER i sang too the fuck-',
        0.5982300263631064, ..., 0.19187065632830977,
        0.10707485138459394, 0.115205

In [0]:
dfnumpy=taskc.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y_train1=dfnumpy[:, 2].reshape(-1, 1)
y_train2=dfnumpy[:, 3].reshape(-1, 1)
y_train3=dfnumpy[:, 4].reshape(-1, 1)

In [0]:
y=np.stack((y_train1.flatten(),y_train2.flatten(),y_train3.flatten()), axis=-1, out=None)

In [12]:
y_train2

array([[0.07610979866070268],
       [0.3673629474131143],
       [0.2685740368872388],
       ...,
       [0.43151670865663655],
       [0.16155983352929942],
       [0.15227155079903518]], dtype=object)

In [13]:
y[0:5]

array([[0.8334315660070198, 0.07610979866070268, 0.10776452321727449],
       [0.48106237178573397, 0.3673629474131143, 0.13884137991817902],
       [0.4388127663192773, 0.2685740368872388, 0.3775726696311655],
       [0.7129954339515537, 0.12350408671699416, 0.11113008071093726],
       [0.6914140593727042, 0.14672323837904971, 0.19228166422122278]],
      dtype=object)

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import re
import string
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
stopwords=set(stopwords.words('english'))
def preprocessinglib(arrt):
    ans =[]
    for txt in arrt:
        unique_words = dict.fromkeys(txt.split())
        txt2=' '.join(unique_words)
        tokens=tknzr.tokenize(txt2)
        ctokens=[]
        for t in tokens:
          if  t not in stopwords and t not in string.punctuation :
            ctokens.append(t)
        #cs=ctokens[0:max_seq_length]
        clean_s = ' '.join(ctokens)
        ans.append(clean_s)
               
    return ans


In [0]:
preprocessedTweets=preprocessinglib(x[:,0])

In [0]:
from sklearn.utils import shuffle
preprocessedTweets,y=shuffle(preprocessedTweets,y)

In [0]:
# Import the dependencies
from keras.models import Sequential
from keras.layers import Dense, LSTM,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [0]:
x_train=preprocessedTweets
y_train=y

In [20]:
y_train

array([[0.32970500650729356, 0.17999333857533661, 0.5282886149584926],
       [0.7917002313617327, 0.09223190703485422, 0.11158894406489282],
       [0.17462440898700485, 0.665503975441615, 0.20898024553828357],
       ...,
       [0.5362114789259027, 0.21174702781058236, 0.3051222130387538],
       [0.553953214916959, 0.14026104093205793, 0.3430198312192558],
       [0.5924243074190462, 0.282917964711436, 0.20051719942393592]],
      dtype=object)

In [0]:

#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a','subtask_b','subtask_c']
englishdata = pd.read_csv("/content/drive/My Drive/EnglishDatac/olid-training-v1.0.tsv", delimiter='\t',names=headers,low_memory=False)
englishdata=englishdata[['id','tweet','subtask_c']]
englishdata=englishdata[1:]

In [0]:
englishdata = englishdata.dropna(subset=['subtask_c'])

In [23]:
englishdata.head()

,id,tweet,subtask_c
2,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,IND
6,97670,@USER Liberals are all Kookoo !!!,OTH
8,52415,@USER was literally just talking about this lo...,GRP
10,13384,@USER Canada doesn’t need another CUCK! We alr...,IND
13,28414,@USER you are a lying corrupt traitor!!! Nobod...,IND


In [0]:
import tensorflow as tf

In [0]:
dfnumpy=englishdata.to_numpy()
x_test=dfnumpy[:, 1].reshape(-1, 1)
y_test=dfnumpy[:, 2].reshape(-1, 1)

In [26]:
y_test

array([['IND'],
       ['OTH'],
       ['GRP'],
       ...,
       ['GRP'],
       ['IND'],
       ['OTH']], dtype=object)

In [27]:
len(y_test.flatten())

3876

In [28]:
len(x_test.flatten())

3876

In [0]:
class InputExample(object):
    def __init__(self, guid, text_a, text_b=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [0]:
def create_tokenizer_from_hub_module():
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [0]:
def convert_single_example(tokenizer, example, max_seq_length=256):
    
   
    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label


In [0]:
x=[]


In [0]:

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels),
    )

In [0]:

def convert_text_to_examples(texts, labels):
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [35]:
tokenizer = create_tokenizer_from_hub_module()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="first",
        bert_path="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1",
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.pooling = pooling
        self.bert_path = bert_path
        if self.pooling not in ["first", "mean"]:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_path, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        if self.pooling == "first":
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
            trainable_layers = ["pooler/dense"]

        elif self.pooling == "mean":
            trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
            trainable_layers = []
        else:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        if self.pooling == "first":
            pooled = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "pooled_output"
            ]
        elif self.pooling == "mean":
            result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result, input_mask)
        else:
            raise NameError(f"Undefined pooling type (must be either first or mean, but is {self.pooling}")

        return pooled

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)


In [0]:
# Build model
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=3, pooling="first")(bert_inputs)
    dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
    pred = tf.keras.layers.Dense(3, activation='softmax')(dense)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [0]:
preprocessedTweets1=preprocessinglib(x_test[:,0])

In [0]:
def giveLabel2(y1):
  i=0
  largest_index=0
  #label1=['IND','GRP','OTH']
  ylabels=[]
  for r in y1:
    if (r=="IND"): 
        largest_index = 0 
    elif (r=="GRP"):
        largest_index = 1 
    else: 
        largest_index = 2 
    
    ylabels.append(largest_index)
    i=i+1;
  return ylabels

In [0]:
yz=giveLabel2(y_test)

In [44]:
set(yz)

{0, 1, 2}

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(yz)
encoded_Y = encoder.transform(yz)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [0]:
ytestlabels=dummy_y

In [47]:
y_train

array([[0.32970500650729356, 0.17999333857533661, 0.5282886149584926],
       [0.7917002313617327, 0.09223190703485422, 0.11158894406489282],
       [0.17462440898700485, 0.665503975441615, 0.20898024553828357],
       ...,
       [0.5362114789259027, 0.21174702781058236, 0.3051222130387538],
       [0.553953214916959, 0.14026104093205793, 0.3430198312192558],
       [0.5924243074190462, 0.282917964711436, 0.20051719942393592]],
      dtype=object)

In [0]:
train_examples = convert_text_to_examples(x_train, y_train)
test_examples = convert_text_to_examples(preprocessedTweets1, ytestlabels)

In [49]:
len(train_examples)

188973

In [0]:
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)


In [0]:
train_labels

In [0]:
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [0]:
test_labels

In [0]:

model = build_model(max_seq_length)

# Instantiate variables
initialize_vars(sess)

model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=1,
    batch_size=32
)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
bert_layer_9 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]          

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def f1_calc(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).to('cpu').numpy().flatten()
  labelsFlattend=np.argmax(predictions,axis=1).to('cpu').numpy().flatten()
  print(predictions)
  print("Predictions Argmax",predictionArgmax)
  print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro')

In [0]:
len(y_test)

In [0]:
#f1 inspired from 
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

In [0]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 64

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [0]:
x_test

In [0]:
preprocessedTweets1=preprocessinglib(x_test[:,0])

In [0]:
X_1 = tokenizer.texts_to_sequences(preprocessedTweets1)
X_test = pad_sequences(X_1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

In [0]:
def giveLabel2(y1):
  i=0
  largest_index=0
  #label1=['IND','GRP','OTH']
  ylabels=[]
  for r in y1:
    if (r=="IND"): 
        largest_index = 0 
    elif (r=="GRP"):
        largest_index = 1 
    else: 
        largest_index = 2 
    
    ylabels.append(largest_index)
    i=i+1;
  return ylabels

In [0]:
yz=giveLabel2(y_test)

In [0]:
set(yz)

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(yz)
encoded_Y = encoder.transform(yz)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [0]:
dummy_y

In [0]:
pip install h5py

In [0]:
encoder.classes_

In [0]:

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("SubtaskCEnglishData.h5")

In [0]:
from keras.models import model_from_json


In [0]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("SubtaskCEnglishData.h5")
print("Loaded model from disk")

In [0]:
score= model.evaluate(X_test,dummy_y)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(score[0],score[1]))

In [0]:
yvalid=model.predict_classes(X_test)

In [0]:
type(X_test)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=predictions.flatten()
  labelsFlattend=labels.flatten()
  #print("Predictions Argmax",predictionArgmax)
  #print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)



In [0]:
f11,acc=calculateF1Score(yvalid,encoded_Y)

In [0]:
acc

In [0]:
f11

In [0]:
ygiven=[]
ypredicted=[]

In [0]:
!unzip -P ****** -qq '/content/drive/My Drive/EnglishDatac/test_c_baseline.csv.zip' 

In [0]:
!unzip -P ****** -qq '/content/drive/My Drive/EnglishDatac/public_data_task_C.zip'

In [0]:
def readData1():
  headers=['id','ypredicted']
  greekdataBaseline = pd.read_csv("test_c_baseline.csv", delimiter=',',names=headers)
  #,converters={"id":convertToInt}       
  #greekdataBaseline.id = greekdataBaseline.id.astype(int)
  #greekdataBaseline=greekdataBaseline[1:]
  print(greekdataBaseline.head())
  print(greekdataBaseline.shape)

  headers=['id','tweet']
  greekDataTest = pd.read_csv("test_c_tweets.tsv", delimiter='\t',names=headers)
                                #converters={"id":convertToInt})
  #greekDataTest=greekDataTest[1:]
  print(greekDataTest.head())
  #print(greekDataTest.dtypes)
  print(greekDataTest.shape)
  result = pd.merge(greekDataTest, greekdataBaseline, on='id', how='inner')
  print(result.head())
  print(result.dtypes)
  print(result.shape)

  #result=
  #result.sort_values(by=['id'], inplace=True)
  #print(result.head())
  dfnumpy=result.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  y=dfnumpy[:, 2].reshape(-1, 1)
  tid=dfnumpy[:, 0].reshape(-1, 1)
  preprocessedTweets=X[:,0]
  return preprocessedTweets,y,tid

In [0]:
preprocessedTweets,ypredict1,tid=readData1()

In [0]:
type(preprocessedTweets)

In [0]:
preprocessedTweets=preprocessinglib(preprocessedTweets)

In [0]:
Xpredict = tokenizer.texts_to_sequences(preprocessedTweets)
Xpredict = pad_sequences(Xpredict, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', Xpredict.shape)

In [0]:
ynew = model.predict_classes(Xpredict)

In [0]:
yz=giveLabel2(ypredict1)

In [0]:
set(yz)

In [0]:
encoded_Ypredict = encoder.transform(yz)


In [0]:
f11,acc=calculateF1Score(ynew,encoded_Ypredict)

In [0]:
f11

In [0]:
acc

In [0]:
print(ynew.flatten().tolist().count(0))
print(ynew.flatten().tolist().count(1))
print(ynew.flatten().tolist().count(2))


In [0]:
yans=ynew.flatten().tolist()

In [0]:
yans3=[]
#yans3.append("IND")

In [0]:
for i in range(len(yans)):
  if yans[i]==0:
    yans3.append("IND")
  elif yans[i]==1:
    yans3.append("GRP")
  else:
    yans3.append("OTH")


In [0]:
tid.shape

In [0]:
#z={'tweet':sentence_predict[],'subtask_a':y_predict}
#print(len(z))
C = {'id': tid.flatten(),
        'predicted': yans3
    }
df = pd.DataFrame(C)
#print (df[df['id'] == 2707] )

export_csv = df.to_csv ('/content/drive/My Drive/EnglishDatac/EnglishSubtaskCAnswerFinalOneHot.csv', index = None, header=True)

